## Tutorial on convolution, using Python

This covers the fMRI-relevant case of convolving a 1-dimensional time-course with a haemodynamic response function (HRF).

Written by Rajeev Raizada: rajeev dot raizada at gmail dot com
 
There is also a follow-up file about the structure of an fMRI-analysis design
matrix: [design_matrix_tutorial.ipynb](https://colab.research.google.com/drive/1JxT6UsrLRbKsY-QG-1w--lUUE3UD26PK#sandboxMode=true)

Neither file assumes any prior knowledge of linear algebra.

This is a live Jupyter notebook, running on the Google Colab server (for free!). So, you can run the code, make changes to it, and see what happens. A good exercise is to change the onset times of the stimuli, and see what that does to the output of the convolution.

You can run each code cell by clicking on the [ ] part to its left. When you mouse-over this, it will turn into an arrow, meaning "Run". Click on that arrow to run that cell. 

You can also run each cell in this, or any other, Jupyter notebook by clicking inside it and hitting the "Shift"+"Enter" keys together.

The easiest way to run all the cells in one go is to click on the "Runtime" menu at the top of this page, and hit "Run all".

## Convolution

In hrf_tutorial.ipynb, we talked about a stimulus "kicking off" an HRF, and the HRFs just adding up on top of each other. The actual math of this is just multiplying and adding. The multiplying part is: we go along the stimulus-time-series vector, element by element, and we multiply each element by the HRF. e.g. if the vector starts off: [ 0, 0, 1, 0, ... ] then we take multiply: 0* HRF, then 0* HRF, then 1* HRF, then 0* HRF etc.

We then take those multiplied HRFs and add them all together, to make an output vector that is the result of the convolution.

The only extra consideration is that each HRF gets "kicked off", i.e. multiplied and added, at a different position along the stimulus-time-series vector. In our [ 0, 0, 1, 0 ] example above, the 1 is in the third position of the stimulus-time-series vector, so we want our 1* HRF to get added onto the same position of our output vector, i.e. starting at the third position.

First, we import the Python modules that we need.

In [0]:
import numpy as np
from matplotlib import pyplot as plt

# This next line just makes the fonts bigger. Their default size is too small
plt.rcParams.update({'font.size': 12})

To make things easier, let's explore convolving with a small HRF, that we talk more about in design_matrix_tutorial.ipynb

We are going to make a NumPy array, which is just like a normal list, except that you can do math with it.

In [0]:
hrf = np.array([ 0,  4,  2,  -1,  0 ])

Let's display this on the screen, using the print() function.

First we show the name of the variable, by putting single quotes around its name, to tell Python that it is a string. Then we start a new line by printing the special character '\n', which is the symbol for "newline". Then we print out the actual value of the variable, simply by passing it as the next input to print().

In [0]:
print('hrf','\n',hrf)

And let's start with a simple stimulus-time-series vector, corresponding to a single stimulus coming on at the second time-point (which will be the second TR, in the case of fMRI).

Note that in Python we start counting at zero. So, the second time-point is actually the third element in our list, because the (human-counting) first element gets counted by Python as the zero-th element! This may seem weird, but it will actually turn out to be useful soon, below.

In [0]:
stim_time_series = np.array([ 0, 0, 1, 0, 0 ])

print('stim_time_series','\n',stim_time_series)

Now, we want to make an output vector that is the result of convolving the stimulus-time-series vector with the HRF. So, we go along the stimulus-time-series vector, element by element, multiply each element by the HRF, and add the result to the corresponding position of our output vector.

`0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `1* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
`0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
`0* [ 0 , 4 , 2 ,-1 , 0 ]`

Let's see how this would all add up for making the (Python-counting) fourth element in our output:

`0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `1* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
`0* [ 0 , 4 , 2 ,-1 , 0 ] +`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
`0* [ 0 , 4 , 2 ,-1 , 0 ]`

<br>
This makes the result: 0* 0 + 0* -1 + 1* 2 +0* 4 + 0* 0 = 2

In the example above, the 4th element of our output vector is equal to:

0th-element of the stimulus-time-series * 4th-element of HRF  + \
1st-element of the stimulus-time-series * 3rd-element of HRF  + \
2nd-element of the stimulus-time-series * 2nd-element of HRF  + \
3rd-element of the stimulus-time-series * 1st-element of HRF  + \
4th-element of the stimulus-time-series * 0th-element of HRF 

As an equation, this is:

Output(pos_in_output) = \
Sum[ with pos_in_hrf going from 0 to length(hrf) ] of \
stim_time_series(pos_in_output - pos_in_hrf) * hrf(pos_in_hrf)

Textbooks will often use a notation like this:

Output(n) means the n-th element of the output of the result of our convolution. 

Typically people write the vector to be convolved as x. In our case, x is stim_time_series. So, x(n) means the n-th element of the onsets-vector, x.

People often call the vector that this gets convolved with the "filter". Another name for it is the "impulse response function", which is where the phrase "hemodynamic response function" comes from.

In our case, the filter is the HRF.

People often call our position in the HRF "k", and the length of the HRF "M".

In that notation:

Output(n) = Sum[ with k going from 0 to M ] x(n-k) * hrf(k)

By the way, note that this means that the result of convolving stim_time_series  with hrf is longer than either stim_time_series or hrf.

length(output) = length(stim_time_series) + length(hrf) - 1

This makes sense when you think of it in terms of a stimulus kicking off an HRF that runs to completion. Even if the stimulus is right at the very end of a scan and if we stop the scan immediately after showing the stimulus, it will still have kicked-off an HRF inside the subject's head.
 
The result of the convolution is the overall HRF-response that the stimuli will evoke, and so if the last stimulus kicks off a fresh HRF, that will continue by one HRF-length past the end of the stimulus-time-series.

By the way, in Python, the command for getting the length of a list is actually len(), not length(), so we'll write len() from now on.

## Using a for-loop to implement the convolution

So, now we have the basic equation for the convolution:

Output(pos_in_output) =

Sum[ with pos_in_hrf going from 0 to length(hrf) ] of stim_time_series(pos_in_output - pos_in_hrf) * hrf(pos_in_hrf)

Now we just need to write a computer program that will plug the actual numbers into that equation and see what they add up to.

In order to do that, we want to go along the each position in the output, one position at a time, and then go along the positions in the HRF, carrying out the multiplication: 

stim_time_series(pos_in_output - pos_in_hrf) * hrf(pos_in_hrf) 

for each position in the HRF, and then adding up the results of all of these multiplications.

In Python, and in almost any programming language, the easiest way to go along a series of positions in a list, one step at a time, is using a for-loop. (A for-loop isn't always the most elegant or most efficient method, but it's often the simplest way, so that's how we'll do it here).

Here's how we'd write a for-loop to do the convolution.

Start off by initialising an output vector all zeros, with one row and (len(stim_time_series) + len(hrf) - 1) columns 

In [0]:
output = np.array([0]*len(stim_time_series) + [0]*(len(hrf)-1))
print(output)

## Building the convolution out of for-loops

This next section tries to explain from scratch how a for-loop works, so my apologies for the redundancy to those of you who already know how to do that. This tutorial is intended to be accessible to complete beginners.

We are going to build two loops. The main loop goes along positions in the output vector. Nested inside that, we have a smaller loop that goes along positions in the HRF. 

We will want to do summing as we move along the positions in the HR.F This involves looking back to the earler positions of stim_time_series, back to position (pos_in_output - pos_in_hrf).

However, if we're at the beginning of the stim_time_series, then these earlier positions don't exist, so they don't play any role in the convolution. In Python, trying to access non-existent parts of the stim_time_series vector would cause an error, so first we check that we're far enough along. Similarly, we can't try to access elements in stim_time_series that happen after the end of that vector.



In [0]:
for pos_in_output in range(0,len(output)):
  # Go around length(output) times, with the value of 
  # pos_in_output starting at 0, and increasing by 1 
  # each time we go around the loop.

  for pos_in_hrf in range(0,len(hrf)):
    # This loop is completely inside the pos_in_output loop. 
    # So, for any given pos_in_output value, we loop through
    # all the positions in the HRF, going from 0 to len(hrf), 
    # which is the last element in the HRF.
    
    pos_in_stim_time_series = pos_in_output - pos_in_hrf

    # Check that our position in the time series isn't
    # falling out of range, either by being less than 0
    # or by having gone past the end
    if (pos_in_stim_time_series >= 0) & \
       (pos_in_stim_time_series < len(stim_time_series)): 
      
      # Multiply the time-series element by the HRF element,
      # and add that to this position of the output vector
      output[pos_in_output] = output[pos_in_output] + \
          stim_time_series[pos_in_stim_time_series] * hrf[pos_in_hrf]     

Let's display the output vector in Python, using the print() command.

In [0]:
print(output)

We can check that this is correct by using NumPy's built-in convolution function. This should be the same as the output above!


In [0]:
output_from_python_conv_function = np.convolve(stim_time_series,hrf)

print(output_from_python_conv_function)

Ok, that seemed to work. Now let's try the same code with a slightly less trivial convolution, and let's plot the results.

Instead of just having one stimulus, at time t=2, we'll have several stimuli, and randomly jittered times.

In [0]:
stim_time_series = np.array([0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0])

output = np.array([0]*len(stim_time_series) + [0]*(len(hrf)-1))

Below is exactly the same code as above, but with some of the lengthier comments taken out, to save repetition. 

In [0]:
for pos_in_output in range(0,len(output)):

  for pos_in_hrf in range(0,len(hrf)):
    
    pos_in_stim_time_series = pos_in_output - pos_in_hrf

    if (pos_in_stim_time_series >= 0) & \
       (pos_in_stim_time_series < len(stim_time_series)): 
      
      output[pos_in_output] = output[pos_in_output] + \
          stim_time_series[pos_in_stim_time_series] * hrf[pos_in_hrf]     

Let's check again that our home-made convolution is correct, by comparing its to using NumPy's built-in convolution function. The two should be the same!

In [0]:
print('Output from home-made conv function','\n',output)

output_from_python_conv_function = np.convolve(stim_time_series,hrf)
print('output_from_python_conv_function','\n',output_from_python_conv_function)

In [0]:
plt.figure(figsize=(8, 8))   # This makes the figure bigger: 8x8

plt.subplot(3,1,1) # This is just to make the plots line up prettily
plt.stem(stim_time_series,label='Stimulus time-series to be convolved')
plt.legend()
plt.axis([0, 23, 0, 1.8]) # This just sets the display graph axis size
plt.ylabel('Stimulus present / absent')
plt.grid(color='k',linestyle=':')

plt.subplot(3,1,2)
plt.plot(output,'rx-',label='Convolved with HRF using home-made for-loop')
plt.legend()
plt.axis([0, 23, -2, 12])
plt.ylabel('Predicted fMRI signal')
plt.grid(color='k',linestyle=':')

plt.subplot(3,1,3)
plt.plot(output_from_python_conv_function,'k*-', \
         label='Convolved with HRF using built-in NumPy function')
plt.legend()
plt.axis([0, 23, -2, 12])
plt.ylabel('Predicted fMRI signal')
plt.grid(color='k',linestyle=':')

